In [1]:
import tkinter as tk
import numpy as np
import random
import time
from queue import Queue, PriorityQueue

# Initialize global variables
GRID_SIZE = 20
CANVAS_SIZE = 600
BUTTON_HEIGHT = 50

# Maze sizes for small, medium, and large
SMALL_MAZE = 10
MEDIUM_MAZE = 20
LARGE_MAZE = 30

class MazeSolverApp:
    def __init__(self, root):
        self.root = root
        self.root.title("AI Maze Solver")

        # Create canvas
        self.canvas = tk.Canvas(self.root, width=CANVAS_SIZE, height=CANVAS_SIZE, bg='white')
        self.canvas.grid(row=0, column=0, columnspan=6)

        # Add control buttons
        self.random_button = tk.Button(self.root, text="Random Maze", command=self.generate_random_maze)
        self.random_button.grid(row=1, column=0)

        self.dfs_button = tk.Button(self.root, text="Solve with DFS", command=self.solve_with_dfs)
        self.dfs_button.grid(row=1, column=1)

        self.bfs_button = tk.Button(self.root, text="Solve with BFS", command=self.solve_with_bfs)
        self.bfs_button.grid(row=1, column=2)

        self.astar_button = tk.Button(self.root, text="Solve with A*", command=self.solve_with_astar)
        self.astar_button.grid(row=1, column=3)

        # Replace IDDFS with Bidirectional Search
        self.bidirectional_button = tk.Button(self.root, text="Solve with Bidirectional", command=self.solve_with_bidirectional)
        self.bidirectional_button.grid(row=1, column=4)

        self.size_label = tk.Label(self.root, text="Maze Size:")
        self.size_label.grid(row=1, column=5)

        self.size_var = tk.StringVar(value="Small")
        self.size_menu = tk.OptionMenu(self.root, self.size_var, "Small", "Medium", "Large", command=self.change_size)
        self.size_menu.grid(row=1, column=6)

        # Initial maze size and maze generation
        self.maze_size = SMALL_MAZE
        self.maze = self.generate_maze(self.maze_size)
        self.solution_path = None
        self.explored_path = []
        self.draw_maze()

    def change_size(self, size_text):
        """Changes maze size based on user selection"""
        if size_text == "Small":
            self.maze_size = SMALL_MAZE
        elif size_text == "Medium":
            self.maze_size = MEDIUM_MAZE
        else:
            self.maze_size = LARGE_MAZE

        self.generate_random_maze()

    def generate_random_maze(self):
        """Generates a random maze and redraws it"""
        self.maze = self.generate_maze(self.maze_size)
        self.solution_path = None
        self.explored_path = []
        self.draw_maze()

    def generate_maze(self, size):
        """Generates a solvable maze by first creating a guaranteed path and then adding random walls."""
        maze = np.ones((size, size), dtype=int)  # Start with a grid full of walls
        start = (0, 0)
        maze[0][0] = 0  # Start point
        maze[size - 1][size - 1] = 0  # End point

        # Step 1: Create a guaranteed path using DFS
        path = []
        visited = set()

        def dfs_generate(x, y):
            path.append((x, y))
            visited.add((x, y))
            maze[x][y] = 0

            if (x, y) == (size - 1, size - 1):
                return True  # Goal reached

            directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
            random.shuffle(directions)

            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 <= nx < size and 0 <= ny < size and (nx, ny) not in visited:
                    if dfs_generate(nx, ny):
                        return True

            return False

        dfs_generate(0, 0)

        # Step 2: Add random walls to the rest of the maze, except for the path
        for i in range(size):
            for j in range(size):
                if (i, j) not in path and random.random() > 0.3:  # 30% chance to be a path, 70% chance to be a wall
                    maze[i][j] = 1  # Wall

        return maze

    def draw_maze(self):
        """Draws the maze and solution path on the canvas"""
        if not self.canvas.winfo_exists():  # Check if canvas exists
            self.canvas = tk.Canvas(self.root, width=CANVAS_SIZE, height=CANVAS_SIZE, bg='white')
            self.canvas.grid(row=0, column=0, columnspan=6)

        self.canvas.delete("all")
        size = len(self.maze)
        cell_size = CANVAS_SIZE // size
        for i in range(size):
            for j in range(size):
                if (i, j) == (0, 0):
                    color = 'green'  # Start point
                elif (i, j) == (size - 1, size - 1):
                    color = 'red'  # End point
                elif self.maze[i][j] == 1:
                    color = 'black'  # Wall
                else:
                    color = 'white'  # Path
                self.canvas.create_rectangle(j * cell_size, i * cell_size, (j + 1) * cell_size, (i + 1) * cell_size, fill=color)

        for (i, j) in self.explored_path:
            self.canvas.create_rectangle(j * cell_size, i * cell_size, (j + 1) * cell_size, (i + 1) * cell_size, fill='blue')
            self.root.update()
            time.sleep(0.02)  # Delay to show exploration

        if self.solution_path:
            # Animate the shortest path turning green
            time.sleep(0.5)  # Pause before turning the path green
            for (i, j) in self.solution_path:
                self.canvas.create_rectangle(j * cell_size, i * cell_size, (j + 1) * cell_size, (i + 1) * cell_size, fill='green')
                self.root.update()
                time.sleep(0.05)

    def solve_with_dfs(self):
        start_time = time.time()
        self.explored_path, self.solution_path = dfs(self.maze)  # Properly track path
        elapsed_time = time.time() - start_time
        self.show_solution(elapsed_time, "DFS")
        time.sleep(1)  # Pause for 1 second before continuing

    def solve_with_bfs(self):
        start_time = time.time()
        self.explored_path, self.solution_path = bfs(self.maze)
        elapsed_time = time.time() - start_time
        self.show_solution(elapsed_time, "BFS")
        time.sleep(1)  # Pause for 1 second before continuing

    def solve_with_astar(self):
        start_time = time.time()
        self.explored_path, self.solution_path = a_star(self.maze)
        elapsed_time = time.time() - start_time
        self.show_solution(elapsed_time, "A*")
        time.sleep(1)  # Pause for 1 second before continuing

    def solve_with_bidirectional(self):
        start_time = time.time()
        self.explored_path, self.solution_path = bidirectional_search(self.maze)
        elapsed_time = time.time() - start_time
        self.show_solution(elapsed_time, "Bidirectional Search")
        time.sleep(1)  # Pause for 1 second before continuing

    def show_solution(self, elapsed_time, algorithm_name):
        if self.solution_path:
            print(f"{algorithm_name} solved the maze in {elapsed_time:.4f} seconds and {len(self.solution_path)} steps.")
            self.draw_maze()
        else:
            print(f"{algorithm_name} could not find a solution.")

# Helper function to reconstruct the shortest path
def reconstruct_path(parent, start, end):
    path = [end]
    while end != start:
        end = parent[end]
        path.append(end)
    path.reverse()  # Reverse the path to get it from start to end
    return path

# DFS function with parent tracking for shortest path
def dfs(maze):
    start = (0, 0)
    end = (len(maze) - 1, len(maze) - 1)
    stack = [start]
    visited = set()
    parent = {}  # Dictionary to track the parent of each node
    explored_path = []
    
    while stack:
        current = stack.pop()
        explored_path.append(current)
        
        if current == end:
            return explored_path, reconstruct_path(parent, start, end)
        
        if current not in visited:
            visited.add(current)
            for neighbor in get_neighbors(current, maze):
                if neighbor not in visited:
                    stack.append(neighbor)
                    parent[neighbor] = current  # Track parent
    
    return explored_path, None

# BFS function with parent tracking for shortest path
def bfs(maze):
    start = (0, 0)
    end = (len(maze) - 1, len(maze) - 1)
    queue = Queue()
    queue.put(start)
    visited = set([start])
    parent = {}  # Dictionary to track the parent of each node
    explored_path = []
    
    while not queue.empty():
        current = queue.get()
        explored_path.append(current)
        
        if current == end:
            return explored_path, reconstruct_path(parent, start, end)
        
        for neighbor in get_neighbors(current, maze):
            if neighbor not in visited:
                queue.put(neighbor)
                visited.add(neighbor)
                parent[neighbor] = current  # Track parent
    
    return explored_path, None

# A* function with parent tracking for shortest path
def a_star(maze):
    start = (0, 0)
    end = (len(maze) - 1, len(maze) - 1)
    pq = PriorityQueue()
    pq.put((0, start))
    visited = set()
    parent = {}  # Dictionary to track the parent of each node
    g_cost = {start: 0}  # Tracks the cost to reach each node
    explored_path = []
    
    while not pq.empty():
        _, current = pq.get()
        explored_path.append(current)
        
        if current == end:
            return explored_path, reconstruct_path(parent, start, end)
        
        if current not in visited:
            visited.add(current)
            for neighbor in get_neighbors(current, maze):
                tentative_g_cost = g_cost[current] + 1
                if neighbor not in g_cost or tentative_g_cost < g_cost[neighbor]:
                    g_cost[neighbor] = tentative_g_cost
                    f_cost = tentative_g_cost + heuristic(neighbor, end)
                    pq.put((f_cost, neighbor))
                    parent[neighbor] = current  # Track parent
    
    return explored_path, None

# Bidirectional Search function with parent tracking for shortest path
def bidirectional_search(maze):
    start = (0, 0)
    end = (len(maze) - 1, len(maze) - 1)
    
    if start == end:
        return [], [start]
    
    # Two queues for the two searches
    queue_start = Queue()
    queue_end = Queue()
    
    queue_start.put(start)
    queue_end.put(end)
    
    # Visited sets for each search
    visited_start = {start}
    visited_end = {end}
    
    # Parent maps to reconstruct paths
    parent_start = {start: None}
    parent_end = {end: None}
    
    # Paths to show explored paths
    explored_path_start = []
    explored_path_end = []
    
    # Helper function to reconstruct the full path
    def reconstruct_full_path(intersect):
        # Reconstruct path from start to intersection
        path_start = []
        current = intersect
        while current is not None:
            path_start.append(current)
            current = parent_start[current]
        path_start.reverse()
        
        # Reconstruct path from end to intersection
        path_end = []
        current = intersect
        while current is not None:
            path_end.append(current)
            current = parent_end[current]
        
        # Remove the intersecting node from the second path to avoid duplication
        path_end = path_end[1:]
        
        # Combine both parts
        full_path = path_start + path_end
        return full_path
    
    # Search alternately from start and end
    while not queue_start.empty() and not queue_end.empty():
        # Explore from the start
        current_start = queue_start.get()
        explored_path_start.append(current_start)
        
        # Explore neighbors from the start
        for neighbor in get_neighbors(current_start, maze):
            if neighbor not in visited_start:
                visited_start.add(neighbor)
                parent_start[neighbor] = current_start
                queue_start.put(neighbor)
                
                # Check if paths meet
                if neighbor in visited_end:
                    full_explored = explored_path_start + explored_path_end
                    return full_explored, reconstruct_full_path(neighbor)
        
        # Explore from the end
        current_end = queue_end.get()
        explored_path_end.append(current_end)
        
        # Explore neighbors from the end
        for neighbor in get_neighbors(current_end, maze):
            if neighbor not in visited_end:
                visited_end.add(neighbor)
                parent_end[neighbor] = current_end
                queue_end.put(neighbor)
                
                # Check if paths meet
                if neighbor in visited_start:
                    full_explored = explored_path_start + explored_path_end
                    return full_explored, reconstruct_full_path(neighbor)
    
    # No path found
    return explored_path_start + explored_path_end, None

# Helper functions
def heuristic(pos, end):
    return abs(pos[0] - end[0]) + abs(pos[1] - end[1])

def get_neighbors(pos, maze):
    row, col = pos
    neighbors = []

    if row > 0 and maze[row - 1][col] == 0:
        neighbors.append((row - 1, col))
    if row < len(maze) - 1 and maze[row + 1][col] == 0:
        neighbors.append((row + 1, col))
    if col > 0 and maze[row][col - 1] == 0:
        neighbors.append((row, col - 1))
    if col < len(maze[0]) - 1 and maze[row][col + 1] == 0:
        neighbors.append((row, col + 1))

    return neighbors

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = MazeSolverApp(root)
    root.mainloop()


DFS solved the maze in 0.0010 seconds and 49 steps.
BFS solved the maze in 0.0004 seconds and 19 steps.
A* solved the maze in 0.0014 seconds and 19 steps.
Bidirectional Search solved the maze in 0.0000 seconds and 19 steps.
